# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff
using PseudoPotentialData

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice = T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
    atoms     = [ElementPsp(:He, pseudopotentials)]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions;
                      functionals=[:lda_x, :lda_c_vwn],
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770828322609                   -0.52    9.0    172ms
  2   -2.772142426381       -2.88       -1.32    1.0    114ms
  3   -2.772170298372       -4.55       -2.52    1.0    101ms
  4   -2.772170670897       -6.43       -3.35    1.0    106ms
  5   -2.772170722235       -7.29       -4.01    2.0    123ms
  6   -2.772170722982       -9.13       -4.90    1.0    108ms
  7   -2.772170723013      -10.51       -5.50    1.0    110ms
  8   -2.772170723015      -11.79       -6.22    2.0    134ms
  9   -2.772170723015      -14.15       -6.72    1.0    116ms
 10   -2.772170723015      -13.64       -7.39    2.0    134ms
 11   -2.772170723015   +  -13.85       -8.03    1.0    117ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770807997169                   -0.52    9.0    182ms
  2   -2.7

1.7735579341557428

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem. This leads to a density-functional perturbation
theory problem, which is automatically set up and solved in the background.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770773614040                   -0.52    9.0    195ms
  2   -2.772059897500       -2.89       -1.33    1.0    100ms
  3   -2.772082857773       -4.64       -2.40    1.0    109ms
  4   -2.772083336262       -6.32       -3.11    1.0    108ms
  5   -2.772083417633       -7.09       -4.64    2.0    126ms
  6   -2.772083417741       -9.97       -4.62    2.0    127ms
  7   -2.772083417809      -10.17       -5.53    1.0    113ms
  8   -2.772083417811      -11.77       -6.61    2.0    131ms
  9   -2.772083417811      -14.45       -7.62    1.0    114ms
 10   -2.772083417811      -14.12       -8.33    2.0    129ms
Solving response problem
Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      13.0   158ms  Non-interacting
   1        0       1       -0.60   